In [1]:
library(pogit)

In [91]:
# block = c(79,137)
# block_simu = c(1215, 1273)
block = c(23, 36)
block_simu = c(164, 177)

In [45]:
X_nointerval <- as.matrix(data.table::fread(paste("/data/GIT/cnv-gene-mapping/data/deletion_simu/block_", block[1], "_" , block[2], 
                                                  "/deletion.genes.block30.for_simu.sample.genes.block_", block[1], "_" , block[2], ".gz", sep = ""), header = F))

In [46]:
dim(X_nointerval)

[1] 13412    14

In [5]:
y <- as.matrix(data.table::fread("/data/GIT/cnv-gene-mapping/data/deletion_simu/deletion.genes.block30.for_simu.sample.y"))

In [6]:
length(y)

[1] 13412

In [5]:
# beta <- as.matrix(data.table::fread("/data/GIT/cnv-gene-mapping/data/deletion_simu/deletion.genes.block30.for_simu.sample.beta"))

In [13]:
# mean(beta); sd(beta); sum(beta == 0); sum(beta == 0)/length(beta)

In [16]:
X <- cbind(rep(1, nrow(X)), X_nointerval)

In [14]:
y <- as.numeric(y[,1])

In [15]:
N <- rep(1, length(y))

In [11]:
# prior <- readRDS("/home/min/GIT/cnv-gene-mapping/data/deletion_simu/deletion.genes.block30.binomial.whole.genome.prior.varbvs.rds")
# prior <- readRDS("/home/min/GIT/cnv-gene-mapping/data/deletion_simu/deletion.genes.block30.binomial.whole.genome.prior.varbvs.nologodds.rds")

In [7]:
prior <- readRDS("/home/min/GIT/cnv-gene-mapping/data/deletion_simu/deletion.genes.block30.binomial.whole.genome.prior.varbvs.nologodds.updateb0.rds")

In [8]:
mu0 = sum(prior$b0 * prior$w)

In [9]:
mu0

[1] 0.7770721

In [10]:
var = sum(prior$sa * prior$w)

In [11]:
var

[1] 0.7117456

In [12]:
pi0 = sum((10^(prior$logodds) / (1 + 10^(prior$logodds))) * prior$w)

In [13]:
pi0

[1] 0.0437755

In [20]:
# b = prior$beta[which(prior$beta != 0)]
# mean(b); var(b); sum(prior$beta == 0); sum(prior$beta == 0)/length(prior$beta)

In [17]:
res <- logitBvs(y, N, X, prior = list(slab = "Normal", aj0 = rep(mu0, ncol(X)-1), V = var, w = c(wa0 = 1, wb0 = 1), pi = c(pa0 = 1.412, pb0 = 10)), 
                mcmc = list(M = 10000, burnin = 2000, msave = TRUE), BVS = FALSE)


MCMC for the binomial logit model:

it = 1 /--- duration of MCMC so far: 29.05 sec.,  expected time to end: Inf  min. 
it = 2 /--- duration of MCMC so far: 29.1 sec.,  expected time to end: 5819.71  min. 
it = 3 /--- duration of MCMC so far: 29.16 sec.,  expected time to end: 2915.91  min. 
it = 4 /--- duration of MCMC so far: 29.25 sec.,  expected time to end: 1949.38  min. 
it = 5 /--- duration of MCMC so far: 29.31 sec.,  expected time to end: 1465.21  min. 
it = 10 /--- duration of MCMC so far: 29.55 sec.,  expected time to end: 656.09  min. 
it = 20 /--- duration of MCMC so far: 30.04 sec.,  expected time to end: 315.71  min. 
it = 50 /--- duration of MCMC so far: 31.25 sec.,  expected time to end: 127.02  min. 
it = 100 /--- duration of MCMC so far: 33.73 sec.,  expected time to end: 67.58  min. 
it = 200 /--- duration of MCMC so far: 38.01 sec.,  expected time to end: 37.57  min. 
it = 500 /--- duration of MCMC so far: 51.22 sec.,  expected time to end: 19.68  min. 
it = 1000 /

In [18]:
saveRDS(res, paste("/home/min/GIT/cnv-gene-mapping/data/deletion_simu/logitBvs.block_", block[1], "_", block[2],".var.selection.res.rds", sep = ""))

In [1]:
# res <- readRDS("/home/min/GIT/cnv-gene-mapping/data/deletion_simu/logitBvs.var.selection.res.rds")

In [30]:
# res1 <- logitBvs(y, N, X, prior = list(slab = "Normal", aj0 = mu, V = 1.725, w = c(wa0 = 1, wb0 = 1), pi = c(pa0 = 0.02, pb0 = 20)), 
#                  mcmc = list(M = 1000, burnin = 200, msave = TRUE), BVS = FALSE)

In [27]:
# saveRDS(res1, "/home/min/GIT/cnv-gene-mapping/data/deletion_simu/logitBvs.novar.selection.nointercept.res.rds")

In [19]:
names(res)

[1] "samplesL"    "data"        "model.logit" "mcmc"        "prior.logit"
 [6] "dur"         "BVS"         "start"       "family"      "call"

In [20]:
names(res$samplesL)

[1] "alpha"       "pdeltaAlpha" "psiAlpha"    "thetaAlpha"  "pgammaAlpha"
[6] "ai"

In [22]:
dim(res$samplesL$alpha)

[1] 12000    15

In [75]:
bhat <- colMeans(res$samplesL$alpha[2001:12000,-1])
bhat1 <- colMeans(res$samplesL$alpha[2001:12000,])

In [76]:
# yhat <- exp(X %*% bhat)/(1+exp(X %*% bhat))
yhat <- X_nointerval %*% bhat
yhat1 <- X %*% bhat1

In [72]:
min(yhat); max(yhat); mean(yhat); median(yhat)

[1] 0

[1] 2.782119

[1] 0.1114029

[1] 0

In [55]:
b_all <- as.matrix(data.table::fread("/data/GIT/cnv-gene-mapping/data/deletion_simu/deletion.genes.block30.for_simu.sample.beta"))
b <- b_all[(block_simu[1]+1):(block_simu[2]+1)]

In [58]:
# y <- exp(X %*% b)/(1+exp(X %*% b))
yb <- X_nointerval %*% b

In [73]:
min(yb); max(yb); mean(yb); median(yb)

[1] 0

[1] 2.924146

[1] 0.1328017

[1] 0

In [99]:
cor(yb, yhat, method = "pearson")

0.9919708


In [83]:
min(yhat1[yhat1<0])

[1] -0.08300953

In [62]:
cor.test(yb, yhat, method=c("pearson"))


	Pearson's product-moment correlation

data:  yb and yhat
t = 908.31, df = 13410, p-value < 2.2e-16
alternative hypothesis: true correlation is not equal to 0
95 percent confidence interval:
 0.9916955 0.9922370
sample estimates:
      cor 
0.9919708 


In [92]:
pdf(file = paste("/home/min/GIT/cnv-gene-mapping/data/deletion_simu/logitBvs.block_", block[1], "_", block[2],".var.selection.beta.hist.pdf", sep = ""))
par(mfrow = c(2,2))
for (i in seq(2, (block[2]-block[1]+2))){
    hist(res$samplesL$alpha[,i], breaks = 80, main = paste("beta", i-1, "histogram"), xlab = paste("beta", i-1))
}
dev.off()

png 
  2

In [94]:
dim(res$samplesL$alpha)

[1] 12000    15

In [79]:
# mean(res$samplesL$alpha[,42])

In [78]:
summary(res)

MCMC for the logit model:

Call:
logitBvs(y = y, N = N, X = X, prior = list(slab = "Normal", aj0 = rep(mu0, 
    ncol(X) - 1), V = var, w = c(wa0 = 1, wb0 = 1), pi = c(pa0 = 1.412, 
    pb0 = 10)), mcmc = list(M = 24000, burnin = 6000, msave = TRUE), 
    BVS = FALSE)


MCMC:
M = 24000 draws after a burn-in of 6000
Thinning parameter: 1

Prior: Normal prior [V=0.711745609189383]

 a0[0]  a0[1]  a0[2]  a0[3]  a0[4]  a0[5]  a0[6]  a0[7]  a0[8]  a0[9] a0[10] 
 0.000  0.777  0.777  0.777  0.777  0.777  0.777  0.777  0.777  0.777  0.777 
a0[11] a0[12] a0[13] a0[14] a0[15] a0[16] a0[17] a0[18] a0[19] a0[20] a0[21] 
 0.777  0.777  0.777  0.777  0.777  0.777  0.777  0.777  0.777  0.777  0.777 
a0[22] a0[23] a0[24] a0[25] a0[26] a0[27] a0[28] a0[29] a0[30] a0[31] a0[32] 
 0.777  0.777  0.777  0.777  0.777  0.777  0.777  0.777  0.777  0.777  0.777 
a0[33] a0[34] a0[35] a0[36] a0[37] a0[38] a0[39] a0[40] a0[41] a0[42] a0[43] 
 0.777  0.777  0.777  0.777  0.777  0.777  0.777  0.777  0.777  0.777  

In [4]:
data(simul_binomial)

In [50]:
y <- simul_binomial$y
N <- simul_binomial$N
X <- as.matrix(simul_binomial[, -c(1,2)])

In [51]:
length(y); length(N); dim(X)

[1] 512

[1] 512

[1] 512  10

In [16]:
head(X)

X.0,X.1,X.2,X.3,X.4,X.5,X.6,X.7,X.8,X.9
1,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,1
1,0,0,0,0,0,0,0,1,0
1,0,0,0,0,0,0,0,1,1
1,0,0,0,0,0,0,1,0,0
1,0,0,0,0,0,0,1,0,1


In [23]:
sum(X[,1])

[1] 512

In [17]:
m1 <- logitBvs(y = y, N = N, X = X)


MCMC for the binomial logit model with variable selection:

it = 1 /--- duration of MCMC so far: 1.02 sec.,  expected time to end: Inf  min. 
it = 2 /--- duration of MCMC so far: 1.03 sec.,  expected time to end: 171.65  min. 
it = 3 /--- duration of MCMC so far: 1.03 sec.,  expected time to end: 86.23  min. 
it = 4 /--- duration of MCMC so far: 1.04 sec.,  expected time to end: 57.7  min. 
it = 5 /--- duration of MCMC so far: 1.04 sec.,  expected time to end: 43.48  min. 
it = 10 /--- duration of MCMC so far: 1.06 sec.,  expected time to end: 19.57  min. 
it = 20 /--- duration of MCMC so far: 1.09 sec.,  expected time to end: 9.54  min. 
it = 50 /--- duration of MCMC so far: 1.16 sec.,  expected time to end: 3.92  min. 
it = 100 /--- duration of MCMC so far: 1.27 sec.,  expected time to end: 2.12  min. 
it = 200 /--- duration of MCMC so far: 1.57 sec.,  expected time to end: 1.29  min. 
it = 500 /--- duration of MCMC so far: 2.32 sec.,  expected time to end: 0.74  min. 
it = 1000 /--

In [18]:
print(m1)

Bayesian variable selection for the binomial logit model:

Call:
logitBvs(y = y, N = N, X = X)

Model: 512 observations
 Covariates: 9
 --- subject to selection: 9
 Random intercept included: no

MCMC:
M = 8000 draws after a burn-in of 2000
BVS started after 1000 iterations
Thinning parameter: 1

Runtime:
total: 101.8 sec.
since burn-in: 85 sec.

In [19]:
summary(m1)

Bayesian variable selection for the binomial logit model:

Call:
logitBvs(y = y, N = N, X = X)


MCMC:
M = 8000 draws after a burn-in of 2000
BVS started after 1000 iterations
Thinning parameter: 1

Prior:  spike-and-slab prior with Student-t slab [V=5]

a0[0] a0[1] a0[2] a0[3] a0[4] a0[5] a0[6] a0[7] a0[8] a0[9] 
    0     0     0     0     0     0     0     0     0     0 
 w[a]  w[b] pi[a] pi[b] 
    1     1     1     1 


Model averaged posterior means, estimated posterior inclusion
 probabilities and 95%-HPD intervals:

            Estimate P(.=1) 95%-HPD[l] 95%-HPD[u]
(Intercept)   -0.452     NA     -0.549     -0.366
alpha.1        0.180  1.000      0.124      0.236
alpha.2       -0.141  1.000     -0.197     -0.086
alpha.3        0.044  0.569      0.000      0.117
alpha.4       -1.077  1.000     -1.135     -1.018
alpha.5       -0.003  0.083     -0.037      0.000
alpha.6        0.000  0.045      0.000      0.000
alpha.7        1.152  1.000      1.096      1.213
alpha.8       -0.077